# Use the atlas website to display your results

The website [http://atlas2.internationalbrainlab.org](http://atlas2.internationalbrainlab.org) enables you to upload and display values per brain region.
This webpage explains how to prepare your data for this purpose, and how to upload them onto the website.

## Install the necessary packages on your machine

- Clone the [`ephys-atlas-web` repository](https://github.com/int-brain-lab/ephys-atlas-web.git)

- If needed, use `conda develop` to ensure this repository is visible from your Python IDE

## Prepare your data

You will have to declare :
- An `acronyms` array containing the acronyms of the brain regions
- One (or more) `feature` array(s) containing the values to be displayed, each corresponding to a brain region listed in the `acronyms` array
- The brain region mapping chosen
- The names of each features

Optional:
- Some text describing each feature
- A hierarchical tree to organise the display of the features


### Default usage: Plot on one hemisphere, using the Beryl mapping

To prepare your data for the visualisation website, we recommend you use the set of brain regions contained in the Beryl mapping.

In [1]:
from ibllib.atlas.regions import BrainRegions
import numpy as np

br = BrainRegions()

mapping = 'Beryl'
beryl_acronyms = br.acronym[np.unique(br.mappings[mapping])]

# Print the list of brain regions contained in this mapping
print(beryl_acronyms)

['void' 'root' 'FRP' 'MOp' 'MOs' 'SSp-n' 'SSp-bfd' 'SSp-ll' 'SSp-m'
 'SSp-ul' 'SSp-tr' 'SSp-un' 'SSs' 'GU' 'VISC' 'AUDd' 'AUDp' 'AUDpo' 'AUDv'
 'VISal' 'VISam' 'VISl' 'VISp' 'VISpl' 'VISpm' 'VISli' 'VISpor' 'ACAd'
 'ACAv' 'PL' 'ILA' 'ORBl' 'ORBm' 'ORBvl' 'AId' 'AIp' 'AIv' 'RSPagl' 'RSPd'
 'RSPv' 'VISa' 'VISrl' 'TEa' 'PERI' 'ECT' 'MOB' 'AOB' 'AON' 'TTd' 'TTv'
 'DP' 'PIR' 'NLOT' 'COAa' 'COAp' 'PAA' 'TR' 'CA1' 'CA2' 'CA3' 'DG' 'FC'
 'IG' 'ENTl' 'ENTm' 'PAR' 'POST' 'PRE' 'SUB' 'ProS' 'HATA' 'APr' 'CLA'
 'EPd' 'EPv' 'LA' 'BLA' 'BMA' 'PA' 'CP' 'ACB' 'FS' 'OT' 'LSc' 'LSr' 'LSv'
 'SF' 'SH' 'AAA' 'BA' 'CEA' 'IA' 'MEA' 'GPe' 'GPi' 'SI' 'MA' 'MS' 'NDB'
 'TRS' 'BST' 'VAL' 'VM' 'VPL' 'VPLpc' 'VPM' 'VPMpc' 'PoT' 'SPF' 'SPA' 'PP'
 'MG' 'LGd' 'LP' 'PO' 'POL' 'SGN' 'Eth' 'AV' 'AM' 'AD' 'IAM' 'IAD' 'LD'
 'IMD' 'MD' 'SMT' 'PR' 'PVT' 'PT' 'RE' 'Xi' 'RH' 'CM' 'PCN' 'CL' 'PF'
 'PIL' 'RT' 'IGL' 'IntG' 'LGv' 'SubG' 'MH' 'LH' 'SO' 'PVH' 'PVa' 'PVi'
 'ARH' 'ADP' 'AVP' 'AVPV' 'DMH' 'MEPO' 'MPO' 'PS' 'PVp' 'PVpo'

/Users/gaelle/Documents/Git/int-brain-lab/ibllib-repo/ibllib/atlas/regions.py:439: RuntimeWarning: invalid value encountered in cast
  level=df_regions.depth.to_numpy().astype(np.uint16),
/Users/gaelle/Documents/Git/int-brain-lab/ibllib-repo/ibllib/atlas/regions.py:441: RuntimeWarning: invalid value encountered in cast
  order=df_regions.graph_order.to_numpy().astype(np.uint16))


Let's imagine you want to plot 3 features (e.g. the mean spike rate, amplitude, or LFP phase), for the following 4 brain regions : VTA, SNc, VISrl and AUDv. The arrays and associated variables would be:

In [3]:
acronyms = np.array(['VTA', 'SNc', 'VISrl', 'AUDv'])

values1 = np.array([1.2, 3.4, 0.0, 0.1])
values2 = np.array([33.5, 234.2, 5.3, 10.6])
values3 = np.array([36.0, 12.3, 34.0, 56.0])

fname1 = 'spike_rate'
fname2 = 'spike_ampl'
fname3 = 'lfp_phase'

# Check that the arrays have the same length
assert len(acronyms) == len(values1)
assert len(acronyms) == len(values2)
assert len(acronyms) == len(values3)

Note: The feature name `fname` should be unique, and can be thought of as a human-readable unique identifier of the feature within the bucket.

### Create a tree
The tree enables you to hierarchically organise the features in your bucket.
If you do not create one (set `tree = None`), on the display all the features will be named by their variable names `fname`, and on the same hierarchy level (i.e. as one drop-down list).
However, you might want to organise features into groups. For example here, we will display the spikes features grouped under "Spike":

In [ ]:
tree = {'Spike': {'mean spike rate (Hz)': fname1,
                  'mean spike amplitude (uV)': fname2},
        'LFP phase (8-10Hz band) (rad)': fname3}


## Upload your data onto the server

Define a bucket name. This bucket name will be used in the webpage URL. One bucket can contain multiple features.
The name of the bucket should ideally easily identify what your bucket will contain, or who it belongs to.
You can follow the convention `my3initials__myproject`, e.g. `sebcha_serotonin`.

Use the `FeatureUploader` to upload your data.

In [ ]:
from server import FeatureUploader

bucket = 'my_bucket_name'

# --------------------------------------------
# Create or load the bucket.
up = FeatureUploader(bucket, tree=tree)

# Create the features.
if not up.features_exist(fname1):
    up.create_features(fname1, acronyms, values1, mapping=mapping)
if not up.features_exist(fname2):
    up.create_features(fname2, acronyms, values2, mapping=mapping)

url = up.get_buckets_url([bucket])
print(url)

The first time you run this, you'll be prompted to enter a global key.
- The global key is: `f9134149-fcb5-4142-a0eb-4e76a0811cf9`

The script should display a URL at the end, click on it to see your features.